In [1]:
import pandas as pd
import numpy as np 
from datetime import datetime

1. https://www.federalreserve.gov/econres/notes/feds-notes/new-way-to-visualize-the-evolution-of-monetary-policy-expectations-20190920.html

2. https://www.atlantafed.org/cenfis/publications/notesfromthevault/1608

3. 

In [12]:
df = pd.read_csv('FFR_rates_futures.csv')
df = df[["Day", "Rate", "PriceNextDay", "PricePrevDay", "PriceDifference", "RateDifference"]]
df = df.drop(range(100,107))